In [1]:
import ecole
import shutil
import os
import sys
import time
import numpy as np
import pyscipopt as pyscip
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.nn import init

from utility import *
from GCN import *

/home/leonaire/anaconda3/envs/ecole/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }

In [3]:
def getInput(scip_parameters,path):
    env = ecole.environment.Branching(\
                observation_function = ecole.observation.NodeBipartite(),scip_params = scip_parameters)
    instance = ecole.scip.Model.from_file(path)
    obs, _, _, _, _ = env.reset(instance)
#     print(obs)
    
    row_features = np.array(obs.row_features.tolist())
    
    original_indice = obs.variable_features[:, -1]
    l = len(original_indice)
    dict_indice = np.zeros((l))
    for i in range(l):
        dict_indice[int(original_indice[i])] = i
    for i in range(l):
        obs.variable_features[i][-1] = dict_indice[i]
    value = obs.variable_features[obs.variable_features[:, -1].argsort()]
    variable_features = np.array(value[:, :-1].tolist())
    
    for i in range(len(obs.edge_features.indices[1])):
        obs.edge_features.indices[1][i] = dict_indice[obs.edge_features.indices[1][i]]

    value_edge_features = np.array(obs.edge_features.values.tolist())
    indice_edge_features = np.array(obs.edge_features.indices.tolist())
#     print(value_edge_features,indice_edge_features)

    nb_cons = row_features.shape[0]
    nb_var = variable_features.shape[0]
    data_cons = np.hstack((np.zeros((nb_cons,19)),row_features))
    data_var = np.hstack((variable_features,np.zeros((nb_var,5))))
    matrix_H = np.vstack((data_var,data_cons))
#     print(matrix_H)

    matrix_A = np.identity(nb_cons+nb_var)
    for i in range(len(value_edge_features)):
        iVar,iCons = indice_edge_features[1][i],indice_edge_features[0][i] 
        matrix_A[iVar][nb_var+iCons] = value_edge_features[i]
        matrix_A[iCons+nb_var][iVar] = value_edge_features[i]
#     print(matrix_A)
    return [matrix_H,matrix_A,nb_var]


In [15]:
def evoluate(model,H,A,nb_var,scip_parameters):
    predictions = net(torch.from_numpy(H).float(),torch.from_numpy(A).float()).squeeze(dim=-1)[:nb_var]
    aux= torch.Tensor([0.5])
    y_hat = (predictions > aux).float() * 1
    return y_hat,predictions

In [16]:
def heuristique(score,A):
    size = len(score)
    size_cons = A.shape[0] - size
    A = A[:size,size_cons:]
    solution = np.zeros((size))
    nb_cover_original = [len(np.where(A[i]==0)[0])for i in range(size)]
    while len(A[0]) > 0:
        nb_cover = [len(np.where(A[i]!=0)[0]) for i in range(size)]
        i_max = 0
        v_max = 0
        for i,v in enumerate(score):
            aux = v * nb_cover[i] / nb_cover_original[i]
            if aux > v_max :
                i_max = i
                v_max = aux
        solution[i_max] = 1
        for i_cons in np.where(A[i_max]!=0)[0][::-1]:
            A = np.delete(A, i_cons, axis=1)
    return solution

In [17]:
def optimize(model,lp_file):
    H,A,nb_var = getInput(scip_parameters,lp_file)
    y_hat,predictions = evoluate(net,H,A,nb_var,scip_parameters)
    res_prect = heuristique(predictions.detach().numpy(),A)
    return res_prect,y_hat

In [18]:
lp = "DataEvoluate/problem800.lp"
train_name = "SC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat"
model_path = "model/"+train_name
criterion = nn.BCELoss()
net = VariablePredictor(24,50,3)
net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
res_her,res_gnn = optimize(net,lp)

In [21]:
print(np.where(res_her == 1)[0])

[ 28  34  43  77  85 101 125 129 139 145 149 192 201 231 240 246 248 260
 288 293 326 356 361 364 368 380 386 392 397]


In [17]:
solver = ecole.scip.Model.from_file(lp)
aspyscip = solver.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.optimize()
res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())


aspyscip.getObjVal()

211.0

In [19]:
print("GNN")
for i,v in enumerate(res_gnn):
    if v != res_scip[i]:
        print(i,v)
print("HER")
for i,v in enumerate(res_her):
    if v != res_scip[i]:
        print(i,v)

GNN
HER
130 0.0
245 0.0


In [20]:
for i,v in enumerate(res_her):
    if v != res_scip[i]:
        print(i,v)
        aspyscip.setSolVal(aspyscip.getBestSol(),aspyscip.getVars()[i],v)


130 0.0
245 0.0


In [21]:
aspyscip.getObjVal()

199.0

In [22]:
for i,v in enumerate(res_gnn):
    if v != res_scip[i]:
        print(i,v,res_scip[i])

In [23]:
(aspyscip.getDualbound() - aspyscip.getObjVal())/min(aspyscip.getDualbound(),aspyscip.getObjVal())

0.06030150753768844

In [ ]:
solver = ecole.scip.Model.from_file(lp)
aspyscip = solver.as_pyscipopt()
aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
aspyscip.getDualbound()